# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

P. Gaikwad  ->  P. Gaikwad  |  ['P. Gaikwad']
F. Nasir  ->  F. Nasir  |  ['F. Nasir']
A. Gould  ->  A. Gould  |  ['A. Gould']
M. Flock  ->  M. Flock  |  ['M. Flock']
H. Linz  ->  H. Linz  |  ['H. Linz']
F. Walter  ->  F. Walter  |  ['F. Walter']
T. Trifonov  ->  T. Trifonov  |  ['T. Trifonov']


Arxiv has 64 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2312.08464


extracting tarball to tmp_2312.08464...

 done.


P. Gaikwad  ->  P. Gaikwad  |  ['P. Gaikwad']
F. Nasir  ->  F. Nasir  |  ['F. Nasir']


...

/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure Figures/ALESS_041.1_fit_subplots_moment_1
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure Figures/ALESS_041.1_fit_subplots_moment_1 as tmp_2312.08959/./Figures/ALESS_041.1_fit_subplots_moment_1.png
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure Figures/ALESS_041.1_fit_subplots_moment_2
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure Figures/ALESS_041.1_fit_subplots_moment_2 as tmp_2312.08959/

Found 118 bibliographic references in tmp_2312.08959/main.bbl.
syntax error in line 235: '=' expected
Retrieving document from  https://arxiv.org/e-print/2312.09156


extracting tarball to tmp_2312.09156...

 done.


Found 66 bibliographic references in tmp_2312.09156/main_text.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.08464-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.08464) | **Constraints on the Evolution of the Ionizing Background and Ionizing  Photon Mean Free Path at the End of Reionization**  |
|| F. B. Davies, et al. -- incl., <mark>P. Gaikwad</mark>, <mark>F. Nasir</mark> |
|*Appeared on*| *2023-12-15*|
|*Comments*| *23 pages, 13 figures, resubmitted to ApJ after referee's comments*|
|**Abstract**| The variations in Ly$\alpha$ forest opacity observed at $z>5.3$ between lines of sight to different background quasars are too strong to be caused by fluctuations in the density field alone. The leading hypothesis for the cause of this excess variance is a late, ongoing reionization process at redshifts below six. Another model proposes strong ionizing background fluctuations coupled to a short, spatially varying mean free path of ionizing photons, without explicitly invoking incomplete reionization. With recent observations suggesting a short mean free path at $z\sim6$, and a dramatic improvement in $z>5$ Ly$\alpha$ forest data quality, we revisit this latter possibility. Here we apply the likelihood-free inference technique of approximate Bayesian computation to jointly constrain the hydrogen photoionization rate $\Gamma_{\rm HI}$ and the mean free path of ionizing photons $\lambda_{\rm mfp}$ from the effective optical depth distributions at $z=5.0$-$6.1$ from XQR-30. We find that the observations are well-described by fluctuating mean free path models with average mean free paths that are consistent with the steep trend implied by independent measurements at $z\sim5$-$6$, with a concomitant rapid evolution of the photoionization rate. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.08819-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.08819) | **The GRAVITY young stellar object survey: XI. Imaging the hot gas  emission around the Herbig Ae star HD 58647**  |
|| Y.-I. Bouarour, et al. -- incl., <mark>M. Flock</mark>, <mark>H. Linz</mark> |
|*Appeared on*| *2023-12-15*|
|*Comments*| *accepted in A&A on 20/11/2023*|
|**Abstract**| We aim to investigate the origin of the HI Br$\gamma$ emission in young stars by using GRAVITY to image the innermost region of circumstellar disks, where important physical processes such as accretion and winds occur. With high spectral and angular resolution, we focus on studying the continuum and the HI Br$\gamma$-emitting area of the Herbig star HD58647. Using VLTI-GRAVITY, we conducted observations of HD58647 with both high spectral and high angular resolution. Thanks to the extensive $uv$ coverage, we were able to obtain detailed images of the circumstellar environment at a sub-au scale, specifically capturing the continuum and the Br$\gamma$-emitting region. Through the analysis of velocity-dispersed images and photocentre shifts, we were able to investigate the kinematics of the HI Br$\gamma$-emitting region. The recovered continuum images show extended emission where the disk major axis is oriented along a position angle of 14\degr. The size of the continuum emission at 5-sigma levels is $\sim$ 1.5 times more extended than the sizes reported from geometrical fitting (3.69 mas $\pm$ 0.02 mas). This result supports the existence of dust particles close to the stellar surface, screened from the stellar radiation by an optically thick gaseous disk. Moreover, for the first time with GRAVITY, the hot gas component of HD58647 traced by the Br$\gamma$ ,has been imaged. This allowed us to constrain the size of the Br$\gamma$-emitting region and study the kinematics of the hot gas; we find its velocity field to be roughly consistent with gas that obeys Keplerian motion. The velocity-dispersed images show that the size of the hot gas emission is from a more compact region than the continuum (2.3 mas $\pm$ 0.2 mas). Finally, the line phases show that the emission is not entirely consistent with Keplerian rotation, hinting at a more complex structure in the hot gaseous disk. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.08959-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.08959) | **The kinematics of massive high-redshift dusty star-forming galaxies**  |
|| A. Amvrosiadis, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2023-12-15*|
|*Comments*| **|
|**Abstract**| We present a new method for modelling the kinematics of galaxies from interferometric observations by performing the optimization of the kinematic model parameters directly in visibility-space instead of the conventional approach of fitting velocity fields produced with the CLEAN algorithm in real-space. We demonstrate our method on ALMA observations of $^{12}$CO (2$-$1), (3$-$2) or (4$-$3) emission lines from an initial sample of 30 massive 850$\mu$m-selected dusty star-forming galaxies with far-infrared luminosities $\gtrsim$$\,10^{12}\,$L$_{\odot}$ in the redshift range $z \sim\,$1.2$-$4.7. Using the results from our modelling analysis for the 12 sources with the highest signal-to-noise emission lines and disk-like kinematics, we conclude the following: (i) Our sample prefers a CO-to-$H_2$ conversion factor, of $\alpha_{\rm CO} = 0.92 \pm 0.36$; (ii) These far-infrared luminous galaxies follow a similar Tully$-$Fisher relation between the circularized velocity, $V_{\rm circ}$, and baryonic mass, $M_{\rm b}$, as more typical star-forming samples at high redshift, but extend this relation to much higher masses $-$ showing that these are some of the most massive disk-like galaxies in the Universe; (iii) Finally, we demonstrate support for an evolutionary link between massive high-redshift dusty star-forming galaxies and the formation of local early-type galaxies using the both the distributions of the baryonic and kinematic masses of these two populations on the $M_{\rm b}\,-\,\sigma$ plane and their relative space densities. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.09156-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.09156) | **The EBLM Project XII. An eccentric, long-period eclipsing binary with a  companion near the hydrogen-burning limit**  |
|| Y. T. Davis, et al. -- incl., <mark>T. Trifonov</mark> |
|*Appeared on*| *2023-12-15*|
|*Comments*| *Resubmitted to MNRAS after positive review*|
|**Abstract**| In the hunt for Earth-like exoplanets it is crucial to have reliable host star parameters, as they have a direct impact on the accuracy and precision of the inferred parameters for any discovered exoplanet. For stars with masses between 0.35 and 0.5 ${\rm M_{\odot}}$ an unexplained radius inflation is observed relative to typical stellar models. However, for fully convective objects with a mass below 0.35 ${\rm M_{\odot}}$ it is not known whether this radius inflation is present as there are fewer objects with accurate measurements in this regime. Low-mass eclipsing binaries present a unique opportunity to determine empirical masses and radii for these low-mass stars. Here we report on such a star, EBLM J2114-39\,B. We have used HARPS and FEROS radial-velocities and TESS photometry to perform a joint fit of the data, and produce one of the most precise estimates of a very low mass star's parameters. Using a precise and accurate radius for the primary star using Gaia DR3 data, we determine J2114-39 to be a $M_1 = 0.987 \pm 0.059$ ${\rm M_{\odot}}$ primary star hosting a fully convective secondary with mass $M_2~=~0.0986~\pm 0.0038~\,\mathrm{M_{\odot}}$, which lies in a poorly populated region of parameter space. With a radius $R_2 =~0.1275~\pm0.0020~\,\mathrm{R_{\odot}}$, similar to TRAPPIST-1, we see no significant evidence of radius inflation in this system when compared to stellar evolution models. We speculate that stellar models in the regime where radius inflation is observed might be affected by how convective overshooting is treated. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2312.08635-b31b1b.svg)](https://arxiv.org/abs/arXiv:2312.08635) | **OGLE-2017-BLG-0448Lb: A Low Mass-Ratio Wide-Orbit Microlensing Planet?**  |
|| R. Zhai, et al. -- incl., <mark>A. Gould</mark> |
|*Appeared on*| *2023-12-15*|
|*Comments*| *submitted to AJ*|
|**Abstract**| The gravitational microlensing technique is most sensitive to planets in a Jupiter-like orbit and has detected more than 200 planets. However, only a few wide-orbit ($s > 2$) microlensing planets have been discovered, where $s$ is the planet-to-host separation normalized to the angular Einstein ring radius, $\theta_{\rm E}$. Here we present the discovery and analysis of a strong candidate wide-orbit microlensing planet in the event, OGLE-2017-BLG-0448. The whole light curve exhibits long-term residuals to the static binary-lens single-source model, so we investigate the residuals by adding the microlensing parallax, microlensing xallarap, an additional lens, or an additional source. For the first time, we observe a complex degeneracy between all four effects. The wide-orbit models with $s \sim 2.5$ and a planet-to-host mass-ratio of $q \sim 10^{-4}$ are significantly preferred, but we cannot rule out the close models with $s \sim 0.35$ and $q \sim 10^{-3}$. A Bayesian analysis based on a Galactic model indicates that, despite the complicated degeneracy, the surviving wide-orbit models all contain a super-Earth-mass to Neptune-mass planet at a projected planet-host separation of $\sim 6$ au and the surviving close-orbit models all consist of a Jovian-mass planet at $\sim 1$ au. The host star is probably an M or K dwarf. We discuss the implications of this dimension-degeneracy disaster on microlensing light-curve analysis and its potential impact on statistical studies. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error bad escape \l at position 9</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2312.08464.md
    + _build/html/tmp_2312.08464/./abc_example.png
    + _build/html/tmp_2312.08464/./xqr30_abc_ghi.png
    + _build/html/tmp_2312.08464/./xqr30_abc_ghi_obs_new.png
    + _build/html/tmp_2312.08464/./xqr30_abc_ghi_models_new.png
exported in  _build/html/2312.08819.md
    + _build/html/tmp_2312.08819/./Continuum_contours3510.png
    + _build/html/tmp_2312.08819/./diff_obs_2.png
    + _build/html/tmp_2312.08819/./BrG_contours_Veloc3510.png
exported in  _build/html/2312.08959.md
    + _build/html/tmp_2312.08959/./Figures/ALESS_class_II_HST_images.png
    + _build/html/tmp_2312.08959/./Figures/ALESS_041.1_fit_subplots_moment_1.png
    + _build/html/tmp_2312.08959/./Figures/ALESS_041.1_fit_subplots_moment_2.png
    + _build/html/tmp_2312.08959/./Figures/ALESS_071.1_fit_subplots_moment_1.png
    + _build/html/tmp_2312.08959/./Figures/ALESS_071.1_fit_subplots_moment_2.png
    + _build/html/tmp_2312.08959/./Figures/ALESS_075.1_fit_subplots_moment_1.png
  

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\url}[1]{\href{#1}{#1}}$
$\newcommand{\dodoi}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{\doeprint}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{\doarXiv}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{\HI}{\mathrm{H I}}$
$\newcommand{\HII}{\mathrm{H II}}$
$\newcommand{\HeI}{\mathrm{He I}}$
$\newcommand{\HeII}{\mathrm{He II}}$
$\newcommand{\HeIII}{\mathrm{He III}}$
$\newcommand{\GHI}{\Gamma_{\HI}}$
$\newcommand{\GHeII}{\Gamma_{\HeII}}$
$\newcommand{\etath}{\eta_{\mathrm{thin}}}$
$\newcommand{\mfp}{\lambda_{\mathrm{mfp}}}$
$\newcommand{\LLS}{\mathrm{LLS}}$
$\newcommand{\lya}{Ly\alpha }$
$\newcommand{\lyb}{Ly\beta }$
$\newcommand{\lyam}{{\mathrm{Ly}\alpha}}$
$\newcommand{\lybm}{{\mathrm{Ly}\beta}}$
$\newcommand{\lyab}{Ly\alpha+\beta}$
$\newcommand{\lyabm}{{\mathrm{Ly}\alpha+\beta}}$
$\newcommand{\teff}{\tau_\mathrm{eff}}$
$\newcommand{\chimp}{Mpc/h}$
$\newcommand{\xhi}{\langle x_{\rm HI}\rangle}$
$\newcommand{\noop}[1]$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# $\bf$ $\large$ Constraints on the Evolution of the Ionizing Background and Ionizing Photon Mean Free Path at the End of Reionization

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2312.08464-b31b1b.svg)](https://arxiv.org/abs/2312.08464)<mark>Appeared on: 2023-12-15</mark> -  _23 pages, 13 figures, resubmitted to ApJ after referee's comments_

</div>
<div id="authors">

F. B. Davies, et al. -- incl., <mark>P. Gaikwad</mark>, <mark>F. Nasir</mark>

</div>
<div id="abstract">

**Abstract:** The variations in Ly $\alpha$ forest opacity observed at $z>5.3$ between lines of sight to different background quasars are too strong to be caused by fluctuations in the density field alone. The leading hypothesis for the cause of this excess variance is a late, ongoing reionization process at redshifts below six. Another model proposes strong ionizing background fluctuations coupled to a short, spatially varying mean free path of ionizing photons, without explicitly invoking incomplete reionization. With recent observations suggesting a short mean free path at $z\sim6$ , and a dramatic improvement in $z>5$ Ly $\alpha$ forest data quality, we revisit this latter possibility. Here we apply the likelihood-free inference technique of approximate Bayesian computation to jointly constrain the hydrogen photoionization rate $\Gamma_{\rm HI}$ and the mean free path of ionizing photons $\lambda_{\rm mfp}$ from the effective optical depth distributions at $z=5.0$ -- $6.1$ from XQR-30. We find that the observations are well-described by fluctuating mean free path models with average mean free paths that are consistent with the steep trend implied by independent measurements at $z\sim5$ -- $6$ , with a concomitant rapid evolution of the photoionization rate.

</div>

<div id="div_fig1">

<img src="tmp_2312.08464/./abc_example.png" alt="Fig7" width="100%"/>

**Figure 7. -** Demonstration of ABC on a mock data set. The blue and red curves in the left panel correspond to the cumulative distribution functions of Ly$\alpha$ forest effective optical depth from a mock XQR-30 data set, with non-detections treated as optimistic ($F=2\times\sigma_F$) or pessimistic ($F=0$), respectively, following \citet{Bosman18,Bosman22}. The shaded regions correspond to the central 95\% of the scatter of additional mock data sets with $\Gamma_{\rm HI}$ and $\lambda_{\rm mfp}$ set to their mean posterior estimates. The grey points in the upper halves of the middle and right panels show the distance metric $\rho$(equation \ref{eqn:abcdist}) computed from 1,000,000 mock data sets. The horizontal lines show three different $\rho$ thresholds below which lie 50\%, 10\%, and 0.1\% of the mock data samples from top to bottom. The corresponding posterior PDFs on $\Gamma_{\rm HI}$ and $\lambda_{\rm mfp}$ are shown in the lower panels, with the true values indicated by the vertical dashed lines. (*fig:abcexample*)

</div>
<div id="div_fig2">

<img src="tmp_2312.08464/./xqr30_abc_ghi.png" alt="Fig2" width="100%"/>

**Figure 2. -** Posterior medians (black circles) and central 68\% credible intervals (black thin error bars) on $\Gamma_{\rm HI}$ from the XQR-30 data set assuming $z_{\rm re}=7.2$. The dark grey shaded region shows the deviation of the posterior medians for $z_{\rm re}=6.7$ and $z_{\rm re}=7.7$, while the light grey shaded region shows the range from more extreme thermal models with $z_{\rm re}=6.2$ and $z_{\rm re}\sim15$. The open orange points show the constraints without the correction for the coarse redshift snapshot sampling (see Appendix \ref{sec:appendix_snaps}). The purple crosses show the posterior medians from the self-consistent model in the $L=100$ Mpc$/h$ hydrodynamical simulation volume, see \S \ref{sec:selfcon}. (*fig:abcghi*)

</div>
<div id="div_fig3">

<img src="tmp_2312.08464/./xqr30_abc_ghi_obs_new.png" alt="Fig9.1" width="50%"/><img src="tmp_2312.08464/./xqr30_abc_ghi_models_new.png" alt="Fig9.2" width="50%"/>

**Figure 9. -** Left: Comparison of our $\Gamma_{\rm HI}$ constraints (black points with error bars, including statistical and systematic uncertainty) to previous measurements from the literature: \citet{Calverley11}(brown diamonds; quasar proximity zone profiles); \citet{WB11}, \citet{BB13}, and \citet{D'Aloisio18}(pink crosses, blue pentagons, and orange triangles; mean Ly$\alpha$ forest transmission); \citet{Davies17}(red squares; Ly$\alpha$ and Ly$\beta$ transmission spikes). No corrections have been made for differences in cosmology or assumptions of the IGM thermal state between these works. The grey points show our constraints with an approximate correction for a bias due to the lack of self-consistency. Right: Comparison to theoretical models of the ionizing background (curves), computed from 1D cosmological radiative transfer calculations by \citet{HM12}(purple), \citet{KS19}(green), \citet{FG20}(blue) and 3D radiation-hydrodynamic cosmological simulations by \citet{Keating19}(brown), \citet{Garaldi22}(orange; THESAN), and \citet{Lewis22}(red; CoDa-III). (*fig:abcghi2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2312.08464"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\HI}{\ion{H}{i}}$
$\newcommand{\HeI}{\ion{He}{i}}$
$\newcommand{\um}{\mum}$
$\newcommand{\brg}{Br\gamma}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\rstar}{R_*}$
$\newcommand{\macc}{\dot{M}_{acc}}$
$\newcommand{\msunyr}{{M}_{\odot}yr^{-1}}$
$\newcommand{\lsun}{L_\odot}$</div>



<div id="title">

# The GRAVITY  young stellar object survey 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2312.08819-b31b1b.svg)](https://arxiv.org/abs/2312.08819)<mark>Appeared on: 2023-12-15</mark> -  _accepted in A&A on 20/11/2023_

</div>
<div id="authors">

Y.-I. Bouarour, et al. -- incl., <mark>M. Flock</mark>, <mark>H. Linz</mark>

</div>
<div id="abstract">

**Abstract:** We aim to investigate the origin of the HI Br$\gamma$ emission in young stars by using GRAVITY to image the innermost region of circumstellar disks, where important physical processes such as accretion and winds occur. With high spectral and angular resolution, we focus on studying the continuum and the HI Br$\gamma$-emitting area of the Herbig star HD58647. Using VLTI-GRAVITY, we conducted observations of HD58647 with both high spectral and high angular resolution. Thanks to the extensive $uv$ coverage, we were able to obtain detailed images of the circumstellar environment at a sub-au scale, specifically capturing the continuum and the Br$\gamma$-emitting region. Through the analysis of velocity-dispersed images and photocentre shifts, we were able to investigate the kinematics of the HI Br$\gamma$-emitting region. The recovered continuum images show extended emission where the disk major axis is oriented along a position angle of 14\degr. The size of the continuum emission at 5-sigma levels is $\sim$ 1.5 times more extended than the sizes reported from geometrical fitting (3.69 mas $\pm$ 0.02 mas). This result supports the existence of dust particles close to the stellar surface, screened from the stellar radiation by an optically thick gaseous disk. Moreover, for the first time with GRAVITY, the hot gas component of HD58647 traced by the Br$\gamma$ ,has been imaged. This allowed us to constrain the size of the Br$\gamma$-emitting region and study the kinematics of the hot gas; we find its velocity field to be roughly consistent with gas that obeys Keplerian motion. The velocity-dispersed images show that the size of the hot gas emission is from a more compact region than the continuum (2.3 mas $\pm$ 0.2 mas). Finally, the line phases show that the emission is not entirely consistent with Keplerian rotation, hinting at a more complex structure in the hot gaseous disk. 

</div>

<div id="div_fig1">

<img src="tmp_2312.08819/./Continuum_contours3510.png" alt="Fig6" width="100%"/>

**Figure 6. -** Results of the image reconstruction. ** Left:** K-band continuum reconstructed images of HD 58647 at different wavelengths (reported in the labels). North is up, and east is to the left. The white hollow ellipse at the lower-left corner of the bottom panel represents the size of the clean beam. Contours represent 3, 5, and 10 $\sigma$ pixel significance levels. ** Middle and right:** Squared visibilities ($\left|V\right|^2$; middle panel) and closure phases (T3$_{\phi}$; right panel)  as a function of the spatial frequency. The observed data with their corresponding error bars are represented as black dots, while the synthetic observables extracted from the reconstructed images are shown in colour. (*contIM*)

</div>
<div id="div_fig2">

<img src="tmp_2312.08819/./diff_obs_2.png" alt="Fig2" width="100%"/>

**Figure 2. -** Example of HR GRAVITY interferometric observations of HD 58647 around the $\HI$\brg line position taken in January 2020 using the (J3-J2-G2-A0) configuration. From top to bottom: HR spectrum (red) and photospheric-corrected spectrum (blue); spectrally dispersed visibility amplitudes; differential phases; and closure phases. Different colours represent different projected baselines and baseline orientations (PAs; visibilities and differential phase panels) or triplets (closure phase panel), as indicated in the middle-top and bottom panel, respectively. (*diffobs*)

</div>
<div id="div_fig3">

<img src="tmp_2312.08819/./BrG_contours_Veloc3510.png" alt="Fig7" width="100%"/>

**Figure 7. -** Results of the image reconstruction. ** Left:**$\HI$\brg line reconstructed images of HD 58647. North is up, and east is to the left. Contours represent 3, 5, and 10 $\sigma$. ** Middle and right:** Comparison of the observed  continuum-subtracted visibility amplitudes and absolute  phases (black circles with error bars) with the synthetic observables extracted from the reconstructed image. (*velomaps*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2312.08819"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\cmark}{\ding{51}}$
$\newcommand{\xmark}{\ding{55}}$
$\newcommand{\gs}{\mathrel{\raise0.35ex\hbox{\scriptstyle >}$
$\lower0.40ex\hbox{{\scriptstyle \sim}}}}$
$\newcommand{\ls}{\mathrel{\raise0.35ex\hbox{\scriptstyle <}$
$\lower0.40ex\hbox{{\scriptstyle \sim}}}}$</div>



<div id="title">

# The  kinematics of massive high-redshift dusty star-forming galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2312.08959-b31b1b.svg)](https://arxiv.org/abs/2312.08959)<mark>Appeared on: 2023-12-15</mark> - 

</div>
<div id="authors">

A. Amvrosiadis, et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We present a new method for modelling the kinematics of galaxies from interferometric observations by performing the optimization of the kinematic model parameters directly in visibility-space instead of the conventional approach of fitting velocity fields produced with the ${\sc clean}$ algorithm in  real-space. We demonstrate our method on ALMA observations of $^{12}$ CO (2 $-$ 1), (3 $-$ 2) or (4 $-$ 3) emission lines from an initial sample of 30 massive 850 $\mu$ m-selected dusty star-forming galaxies with far-infrared luminosities $\gtrsim 10^{12} $ L $_{\odot}$ in the redshift range $z \sim$ 1.2--4.7. Using the results from our modelling analysis for the 12 sources with the highest signal-to-noise emission lines and disk-like kinematics, we conclude the following: (i) Our sample prefers a CO-to- $H_2$ conversion factor,  of $\alpha_{\rm CO} = 0.92 \pm 0.36$ ; (ii) These far-infrared luminous galaxies follow a similar Tully--Fisher relation between the circularized velocity, $V_{\rm circ}$ , and baryonic mass, $M_{\rm b}$ , as more typical star-forming samples at high redshift, but extend this relation to much higher masses -- showing that these are some of the most massive disk-like galaxies in the Universe; (iii) Finally, we demonstrate support for an evolutionary link between massive high-redshift dusty star-forming galaxies and the formation of local early-type galaxies using the both the distributions of the  baryonic and kinematic masses of these two populations on the $M_{\rm b}$ -- $\sigma$ plane and their relative space densities.

</div>

<div id="div_fig1">

<img src="tmp_2312.08959/./Figures/ALESS_class_II_HST_images.png" alt="Fig2" width="100%"/>

**Figure 2. -** HST images (greyscale) of three of the Class \Romannum{2} sources in our sample (ALESS 088.1, 101.1, 112.1). The blue contours show the distribution of the CO emission, while the HST imaging suggests potentially complex morphologies for these sources. (*fig:figure_1_for_Appendix_A*)

</div>
<div id="div_fig2">

<img src="tmp_2312.08959/./Figures/ALESS_041.1_fit_subplots_moment_1.png" alt="Fig1.1" width="16%"/><img src="tmp_2312.08959/./Figures/ALESS_041.1_fit_subplots_moment_2.png" alt="Fig1.2" width="16%"/><img src="tmp_2312.08959/./Figures/ALESS_071.1_fit_subplots_moment_1.png" alt="Fig1.3" width="16%"/><img src="tmp_2312.08959/./Figures/ALESS_071.1_fit_subplots_moment_2.png" alt="Fig1.4" width="16%"/><img src="tmp_2312.08959/./Figures/ALESS_075.1_fit_subplots_moment_1.png" alt="Fig1.5" width="16%"/><img src="tmp_2312.08959/./Figures/ALESS_075.1_fit_subplots_moment_2.png" alt="Fig1.6" width="16%"/>

**Figure 1. -** 
  Moment 1 (velocity; left column) and 2 (dispersion; right column) maps for three example sources in our sample (the name of each source is indicated at the top left corner). From left to right in each row we show the observed data, the maximum likelihood model and residual dirty moment maps (the moment maps were computed after masking any emission below $3\sigma$). In these example sources (as well as for the other sources regarded as well described by our rotating disk model; see Section \ref{sec:model_observations}) the model does a good job at reproducing the observed data. The black dotted line in each panel corresponds to the best-fit position angle, $\theta$, of the major axis. The black ellipse in the bottom left corner represents the beam.  (*fig:figure_2*)

</div>
<div id="div_fig3">

<img src="tmp_2312.08959/./Figures/sims/sims_0.5arcsec.png" alt="Fig3.1" width="50%"/><img src="tmp_2312.08959/./Figures/sims/sims_1.0arcsec.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** Output versus input values for the two main parameters of interest, the maximum rotation velocity ($V_{\rm max}$) and the velocity dispersion ($\sigma$), color-coded by the true inclination. The different columns corresponds to datasets with different SNR, indicated at the top left corner of each panel. The top and bottom figures correspond to datasets with different resolution, 0.5 arcsec and 1.0 arcsec, respectively. (*fig:sims*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2312.08959"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mass}{M_2~=~0.0986~\pm 0.0038~ \mathrm{M_{\sun}}}$
$\newcommand{\radius}{R_2 =~0.1275~\pm0.0020~ \mathrm{R_{\sun}}}$
$\newcommand{\period}{P =~44.92471~\pm0.00025~ \rm{ day}}$
$\newcommand{\loggsec}{\log g_2 = 5.222 \pm 0.0135}$
$\newcommand{\massfunction}{f_m = 0.0008127\pm0.0000028~ \mathrm{M_{\sun}}}$
$\newcommand{\eccentricity}{e = 0.56767 \pm 0.00045}$
$\newcommand{\massratio}{q = 0.0999 \pm 0.0022 }$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The EBLM Project XII.  An eccentric, long-period eclipsing binary with a companion near the hydrogen-burning limit

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2312.09156-b31b1b.svg)](https://arxiv.org/abs/2312.09156)<mark>Appeared on: 2023-12-15</mark> -  _Resubmitted to MNRAS after positive review_

</div>
<div id="authors">

Y. T. Davis, et al. -- incl., <mark>T. Trifonov</mark>

</div>
<div id="abstract">

**Abstract:** In the hunt for Earth-like exoplanets it is crucial to have reliable host star parameters, as they have a direct impact on the accuracy and precision of the inferred parameters for any discovered exoplanet. For stars with masses between 0.35 and 0.5 ${\rm M_{\sun}}$ an unexplained radius inflation is observed relative to typical stellar models. However, for fully convective objects with a mass below 0.35 ${\rm M_{\sun}}$ it is not known whether this radius inflation is present as there are fewer objects with accurate measurements in this regime. Low-mass eclipsing binaries present a unique opportunity to determine empirical masses and radii for these low-mass stars. Here we report on such a star, EBLM J2114-39 B. We have used HARPS and FEROS radial-velocities and _TESS_ photometry to perform a joint fit of the data, and produce one of the most precise estimates of a very low mass star's parameters. Using a precise and accurate radius for the primary star using $_ Gaia_$ DR3 data, we determine J2114-39 to be a $M_1 = 0.987 \pm 0.059$  ${\rm M_{\sun}}$ primary star hosting a fully convective secondary with mass $\mass$ , which lies in a poorly populated region of parameter space. With a radius $\radius$ , similar to TRAPPIST-1, we see no significant evidence of radius inflation in this system when compared to stellar evolution models. We speculate that stellar models in the regime where radius inflation is observed might be affected by how convective overshooting is treated.

</div>

<div id="div_fig1">

<img src="tmp_2312.09156/./Figures/MR_plot.png" alt="Fig3" width="100%"/>

**Figure 3. -** 
    Mass-radius diagram with logarithmic mass axis, showing published single-lined systems (SB1) in grey, double-lined binaries (SB2) in black and EBLM J2114-39 in coral colour. The $1 \rm Gyr$ and $5 \rm Gyr$ isochrones are shown in purple and teal respectively. The lower panel highlights differences between the objects and the $5 \rm Gyr$ isochrone. Black and grey data points from compendium (and cited papers) in [ and Triaud (2020)](https://ui.adsabs.harvard.edu/abs/2020NatAs...4..650T), with additional points from [ and Morales (2009)](https://ui.adsabs.harvard.edu/abs/2009ApJ...691.1400M).
     (*fig:mass_radius_plot*)

</div>
<div id="div_fig2">

<img src="tmp_2312.09156/./Figures/TESS_sec1spocphase.png" alt="Fig1" width="100%"/>

**Figure 1. -** 
    Top panel: Raw _ TESS_ data in purple with 20 random samples of the GP from the MCMC modelling the additional noise in blue. Middle panel: The phase-shifted eclipse of EBLM J2114-39, data shown in purple after the signal modelled by the GP has been removed. Twenty random samples from the MCMC run are displayed in teal colour. Bottom panel: The residuals to the best fit model. (*fig:phased_light_curve*)

</div>
<div id="div_fig3">

<img src="tmp_2312.09156/./Figures/phased_RV.png" alt="Fig2" width="100%"/>

**Figure 2. -** 
    Phased radial-velocity measurements for EBLM J2114-39 from the HARPS (in purple) and FEROS (in pink) spectrographs. A sample of 20 random models from the MCMC are shown in teal colour. The residuals to best fit model are shown in the lower panel. (*fig:phased_rv*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2312.09156"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

343  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
